In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import _pickle as cPickle
import os
import s3fs
import boto3
import math
import csv
import re
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn import metrics
s3 = s3fs.S3FileSystem(anon=False)

plt.rcdefaults()

# Read in data

### Huntingtons

In [3]:
hunt_brain_unhealthy_56=pd.read_csv('s3://ewasunhealthy/data/brain/huntingtons/hunt_brain_top_56.csv', index_col=0,low_memory=False)

In [4]:
huntingtons_df=hunt_brain_unhealthy_56.drop(columns=['ch.1.169608016R'])
huntingtons_df['status']=[1]*len(huntingtons_df.index)
cols = list(huntingtons_df.columns.values)
cols=cols[1:-1]
huntingtons_df = huntingtons_df[['status']+['AGE']+cols]
huntingtons_df

,status,AGE,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,cg23606718,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM1871451,1,83.0,0.255,0.432,0.147,0.640,0.179,0.926,0.426,0.159,...,0.147,0.981,0.027,0.166,0.082,0.096,0.078,0.270,0.221,0.571
GSM1871457,1,48.0,0.257,0.554,0.032,0.648,0.173,0.909,0.311,0.112,...,0.158,0.968,0.050,0.126,0.047,0.048,0.030,0.204,0.227,0.571
GSM1871498,1,51.0,0.282,0.248,0.143,0.550,0.244,0.920,0.332,0.168,...,0.210,0.914,0.022,0.170,0.057,0.047,0.049,0.306,0.239,0.603
GSM1871545,1,65.0,0.335,0.470,0.114,0.617,0.178,0.888,0.352,0.178,...,0.146,0.954,0.080,0.177,0.063,0.077,0.051,0.333,0.311,0.597
GSM1871590,1,67.0,0.316,0.586,0.056,0.595,0.189,0.881,0.402,0.160,...,0.174,0.964,0.037,0.189,0.051,0.056,0.040,0.241,0.186,0.575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871815,1,56.0,0.231,0.244,0.117,0.587,0.175,0.919,0.265,0.140,...,0.077,0.959,0.048,0.112,0.052,0.038,0.025,0.161,0.181,0.425
GSM1871849,1,62.0,0.212,0.433,0.125,0.667,0.044,0.845,0.155,0.147,...,0.062,0.969,0.049,0.141,0.066,0.044,0.063,0.205,0.211,0.420
GSM1871852,1,58.0,0.250,0.449,0.120,0.623,0.212,0.917,0.273,0.130,...,0.159,0.935,0.035,0.141,0.069,0.048,0.043,0.233,0.176,0.496
GSM1871860,1,91.0,0.287,0.657,0.129,0.649,0.017,0.902,0.324,0.176,...,0.055,0.983,0.052,0.154,0.069,0.060,0.057,0.276,0.237,0.384


In [5]:
Xhunt = huntingtons_df.iloc[:,2:]
yhunt = huntingtons_df.iloc[:,0]
X_work_hunt, X_work_hunt, y_work_hunt, y_hold_hunt = train_test_split(Xhunt, yhunt, test_size=0.25,random_state=7)
#with open('hunt_working_CpGs', 'wb') as fp:
        #cPickle.dump(list(X_work_hunt.index), fp)
#with open('hunt_held_out_CpGs', 'wb') as fp:
        #cPickle.dump(list(X_work_hunt.index), fp)

### Alzheimer's

In [6]:
alz_brain_unhealthy_56=pd.read_csv('s3://ewasunhealthy/data/brain/alzheimers/alz_brain_top_56.csv', index_col=0,low_memory=False)

In [7]:
alz_df=alz_brain_unhealthy_56.drop(columns=['ch.1.169608016R'])
alz_df['status']=[1]*len(alz_df.index)
cols = list(alz_df.columns.values)
cols=cols[1:-1]
alz_df = alz_df[['status']+['AGE']+cols]
alz_df

,status,AGE,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,cg23606718,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM1068826,1,88.0,0.052,0.512,0.113,0.679,0.010,0.846,0.275,0.078,...,0.023,0.972,0.105,0.044,0.135,0.120,0.029,0.158,0.243,0.407
GSM1068827,1,92.0,0.103,0.519,0.142,0.532,0.015,0.811,0.275,0.030,...,0.018,0.959,0.115,0.125,0.172,0.136,0.052,0.106,0.265,0.444
GSM1068829,1,93.0,0.064,0.620,0.154,0.535,0.024,0.819,0.307,0.065,...,0.015,0.968,0.053,0.193,0.229,0.161,0.032,NaN,0.329,0.480
GSM1068832,1,96.0,0.049,0.549,0.133,0.575,0.019,0.871,0.289,0.028,...,0.009,0.980,0.058,0.290,0.162,0.089,0.024,NaN,0.317,0.422
GSM1068833,1,86.0,0.081,0.287,0.089,0.568,0.021,0.863,0.165,0.044,...,0.015,0.981,0.062,0.189,0.116,0.083,0.022,0.107,0.236,0.431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM2809060,1,80.0,0.364,0.635,0.144,0.692,0.189,0.910,0.407,0.202,...,0.115,0.950,0.069,0.153,0.111,0.083,0.081,NaN,0.184,0.589
GSM2809061,1,70.0,0.366,0.520,0.118,0.554,0.394,0.887,0.383,0.221,...,0.311,0.910,0.075,0.117,0.090,0.058,0.054,NaN,0.214,0.592
GSM2809062,1,75.0,0.328,0.321,0.124,0.610,0.134,0.892,0.400,0.130,...,0.101,0.943,0.031,0.161,0.072,0.056,0.064,0.285,0.209,0.610
GSM2809063,1,79.0,0.356,0.537,0.145,0.668,0.143,0.895,0.390,0.166,...,0.097,0.935,0.051,0.117,0.110,0.054,0.074,NaN,0.220,0.540


In [8]:
Xalz = alz_df.iloc[:,1:]
yalz = alz_df.iloc[:,0]
X_work_alz, X_hold_alz, y_work_alz, y_hold_alz = train_test_split(Xalz, yalz, test_size=0.25,random_state=7)
#with open('alz_working_CpGs', 'wb') as fp:
        #cPickle.dump(list(X_work_alz.index), fp)
#with open('alz_held_out_CpGs', 'wb') as fp:
        #cPickle.dump(list(X_hold_alz.index), fp)

### Healthy Controls

In [9]:
hc_brain_56=pd.read_csv('s3://ewasunhealthy/data/brain/healthy/hc_brain_55CpGs.csv', index_col=0,low_memory=False)

In [10]:
hc_brain_56 = hc_brain_56.rename(columns = {'age': 'AGE'}, inplace = False)

In [11]:
hc_brain_56['status']=[0]*len(hc_brain_56.index)
cols = list(hc_brain_56.columns.values)
cols=cols[1:-1]
hc_brain_56 = hc_brain_56[['status']+['AGE']+cols]
hc_brain_56

,status,AGE,cg00050873,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM2139432,0,71.0000,0.826,0.288,0.331,0.128,0.607,0.106,0.829,0.299,...,0.088,0.948,0.037,0.217,0.105,0.102,0.124,0.289,0.240,0.475
GSM2139249,0,76.0000,NaN,0.329,0.378,0.140,0.563,0.101,0.880,0.413,...,0.095,0.942,0.039,0.161,0.130,0.064,0.084,0.318,0.215,0.466
GSM2139398,0,102.0000,NaN,0.294,0.339,0.067,0.604,0.188,0.824,0.395,...,0.154,0.947,0.044,0.124,0.114,0.085,0.063,0.317,0.219,0.432
GSM2139297,0,108.0000,NaN,0.327,0.528,0.153,0.703,0.054,0.847,0.438,...,0.053,0.964,0.088,0.261,0.168,0.105,0.192,0.419,0.293,0.559
GSM1069208,0,40.0000,0.828,0.152,0.452,0.056,0.559,0.250,0.840,0.212,...,0.232,0.898,0.038,0.083,0.073,0.044,0.023,0.231,0.193,0.602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871753,0,53.0000,NaN,0.215,0.498,0.127,0.624,0.216,0.900,0.133,...,0.194,0.977,0.044,0.094,0.054,0.038,0.048,0.205,0.201,0.740
GSM1438255,0,53.1594,NaN,0.198,0.510,0.049,0.568,0.289,0.878,0.119,...,0.236,0.925,0.029,0.072,0.024,0.022,0.036,0.124,0.172,0.540
GSM1438138,0,53.3855,0.851,0.239,0.329,0.114,0.622,0.292,0.770,0.332,...,0.233,0.932,0.072,0.090,0.046,0.029,0.053,0.203,0.188,0.450
GSM1438147,0,53.4770,0.840,0.239,0.466,0.058,0.682,0.196,0.909,0.301,...,0.226,0.966,0.037,0.162,0.085,0.060,0.074,0.174,0.220,0.524


In [12]:
Xhc = hc_brain_56.iloc[:,1:]
yhc = hc_brain_56.iloc[:,0]
X_work_hc, X_hold_hc, y_work_hc, y_hold_hc = train_test_split(Xhc, yhc, test_size=0.25,random_state=7)

In [13]:
frames=[hc_brain_56,alz_df,huntingtons_df]
#key=healthy_df.columns
result = pd.concat(frames)
result

,status,AGE,cg00050873,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM2139432,0,71.0,0.826,0.288,0.331,0.128,0.607,0.106,0.829,0.299,...,0.088,0.948,0.037,0.217,0.105,0.102,0.124,0.289,0.240,0.475
GSM2139249,0,76.0,NaN,0.329,0.378,0.140,0.563,0.101,0.880,0.413,...,0.095,0.942,0.039,0.161,0.130,0.064,0.084,0.318,0.215,0.466
GSM2139398,0,102.0,NaN,0.294,0.339,0.067,0.604,0.188,0.824,0.395,...,0.154,0.947,0.044,0.124,0.114,0.085,0.063,0.317,0.219,0.432
GSM2139297,0,108.0,NaN,0.327,0.528,0.153,0.703,0.054,0.847,0.438,...,0.053,0.964,0.088,0.261,0.168,0.105,0.192,0.419,0.293,0.559
GSM1069208,0,40.0,0.828,0.152,0.452,0.056,0.559,0.250,0.840,0.212,...,0.232,0.898,0.038,0.083,0.073,0.044,0.023,0.231,0.193,0.602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871815,1,56.0,NaN,0.231,0.244,0.117,0.587,0.175,0.919,0.265,...,0.077,0.959,0.048,0.112,0.052,0.038,0.025,0.161,0.181,0.425
GSM1871849,1,62.0,NaN,0.212,0.433,0.125,0.667,0.044,0.845,0.155,...,0.062,0.969,0.049,0.141,0.066,0.044,0.063,0.205,0.211,0.420
GSM1871852,1,58.0,NaN,0.250,0.449,0.120,0.623,0.212,0.917,0.273,...,0.159,0.935,0.035,0.141,0.069,0.048,0.043,0.233,0.176,0.496
GSM1871860,1,91.0,NaN,0.287,0.657,0.129,0.649,0.017,0.902,0.324,...,0.055,0.983,0.052,0.154,0.069,0.060,0.057,0.276,0.237,0.384


In [14]:
all_brain_55_cpgs=result

In [15]:
all_brain_55_cpgs.to_csv('all_brain_55_cpgs.csv', encoding='utf-8', index=True)

In [16]:
all_test_cpgs=list(y_hold_hunt.index)+list(y_hold_alz.index)+list(y_hold_hc.index)

In [17]:
train_data=all_brain_55_cpgs
train_data

,status,AGE,cg00050873,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM2139432,0,71.0,0.826,0.288,0.331,0.128,0.607,0.106,0.829,0.299,...,0.088,0.948,0.037,0.217,0.105,0.102,0.124,0.289,0.240,0.475
GSM2139249,0,76.0,NaN,0.329,0.378,0.140,0.563,0.101,0.880,0.413,...,0.095,0.942,0.039,0.161,0.130,0.064,0.084,0.318,0.215,0.466
GSM2139398,0,102.0,NaN,0.294,0.339,0.067,0.604,0.188,0.824,0.395,...,0.154,0.947,0.044,0.124,0.114,0.085,0.063,0.317,0.219,0.432
GSM2139297,0,108.0,NaN,0.327,0.528,0.153,0.703,0.054,0.847,0.438,...,0.053,0.964,0.088,0.261,0.168,0.105,0.192,0.419,0.293,0.559
GSM1069208,0,40.0,0.828,0.152,0.452,0.056,0.559,0.250,0.840,0.212,...,0.232,0.898,0.038,0.083,0.073,0.044,0.023,0.231,0.193,0.602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871815,1,56.0,NaN,0.231,0.244,0.117,0.587,0.175,0.919,0.265,...,0.077,0.959,0.048,0.112,0.052,0.038,0.025,0.161,0.181,0.425
GSM1871849,1,62.0,NaN,0.212,0.433,0.125,0.667,0.044,0.845,0.155,...,0.062,0.969,0.049,0.141,0.066,0.044,0.063,0.205,0.211,0.420
GSM1871852,1,58.0,NaN,0.250,0.449,0.120,0.623,0.212,0.917,0.273,...,0.159,0.935,0.035,0.141,0.069,0.048,0.043,0.233,0.176,0.496
GSM1871860,1,91.0,NaN,0.287,0.657,0.129,0.649,0.017,0.902,0.324,...,0.055,0.983,0.052,0.154,0.069,0.060,0.057,0.276,0.237,0.384


In [18]:
all_ids=list(all_brain_55_cpgs.index)
for i in all_ids:
    if i in all_test_cpgs:
        train_data=train_data.drop([i])
        
train_data=train_data.loc[train_data['AGE'] >= 20]
train_data=train_data.loc[train_data['AGE'] <= 110]
train_data

,status,AGE,cg00050873,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM2139432,0,71.0,0.826,0.288,0.331,0.128,0.607,0.106,0.829,0.299,...,0.088,0.948,0.037,0.217,0.105,0.102,0.124,0.289,0.240,0.475
GSM2139249,0,76.0,NaN,0.329,0.378,0.140,0.563,0.101,0.880,0.413,...,0.095,0.942,0.039,0.161,0.130,0.064,0.084,0.318,0.215,0.466
GSM2139398,0,102.0,NaN,0.294,0.339,0.067,0.604,0.188,0.824,0.395,...,0.154,0.947,0.044,0.124,0.114,0.085,0.063,0.317,0.219,0.432
GSM2139297,0,108.0,NaN,0.327,0.528,0.153,0.703,0.054,0.847,0.438,...,0.053,0.964,0.088,0.261,0.168,0.105,0.192,0.419,0.293,0.559
GSM1443489,0,40.0,0.828,0.152,0.452,0.056,0.559,0.250,0.840,0.212,...,0.232,0.898,0.038,0.083,0.073,0.044,0.023,0.231,0.193,0.602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871717,1,42.0,NaN,0.209,0.464,0.056,0.650,0.198,0.846,0.109,...,0.198,0.920,0.041,0.079,0.048,0.028,0.033,0.158,0.194,0.518
GSM1871849,1,62.0,NaN,0.212,0.433,0.125,0.667,0.044,0.845,0.155,...,0.062,0.969,0.049,0.141,0.066,0.044,0.063,0.205,0.211,0.420
GSM1871852,1,58.0,NaN,0.250,0.449,0.120,0.623,0.212,0.917,0.273,...,0.159,0.935,0.035,0.141,0.069,0.048,0.043,0.233,0.176,0.496
GSM1871860,1,91.0,NaN,0.287,0.657,0.129,0.649,0.017,0.902,0.324,...,0.055,0.983,0.052,0.154,0.069,0.060,0.057,0.276,0.237,0.384


In [19]:
train_data.to_csv('brain_55_cpgs_train.csv', encoding='utf-8', index=True)

In [20]:
test_data=all_brain_55_cpgs
test_data

,status,AGE,cg00050873,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM2139432,0,71.0,0.826,0.288,0.331,0.128,0.607,0.106,0.829,0.299,...,0.088,0.948,0.037,0.217,0.105,0.102,0.124,0.289,0.240,0.475
GSM2139249,0,76.0,NaN,0.329,0.378,0.140,0.563,0.101,0.880,0.413,...,0.095,0.942,0.039,0.161,0.130,0.064,0.084,0.318,0.215,0.466
GSM2139398,0,102.0,NaN,0.294,0.339,0.067,0.604,0.188,0.824,0.395,...,0.154,0.947,0.044,0.124,0.114,0.085,0.063,0.317,0.219,0.432
GSM2139297,0,108.0,NaN,0.327,0.528,0.153,0.703,0.054,0.847,0.438,...,0.053,0.964,0.088,0.261,0.168,0.105,0.192,0.419,0.293,0.559
GSM1069208,0,40.0,0.828,0.152,0.452,0.056,0.559,0.250,0.840,0.212,...,0.232,0.898,0.038,0.083,0.073,0.044,0.023,0.231,0.193,0.602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871815,1,56.0,NaN,0.231,0.244,0.117,0.587,0.175,0.919,0.265,...,0.077,0.959,0.048,0.112,0.052,0.038,0.025,0.161,0.181,0.425
GSM1871849,1,62.0,NaN,0.212,0.433,0.125,0.667,0.044,0.845,0.155,...,0.062,0.969,0.049,0.141,0.066,0.044,0.063,0.205,0.211,0.420
GSM1871852,1,58.0,NaN,0.250,0.449,0.120,0.623,0.212,0.917,0.273,...,0.159,0.935,0.035,0.141,0.069,0.048,0.043,0.233,0.176,0.496
GSM1871860,1,91.0,NaN,0.287,0.657,0.129,0.649,0.017,0.902,0.324,...,0.055,0.983,0.052,0.154,0.069,0.060,0.057,0.276,0.237,0.384


In [21]:
all_ids=list(all_brain_55_cpgs.index)
for i in all_ids:
    if i not in all_test_cpgs:
        test_data=test_data.drop([i])
        
test_data

,status,AGE,cg00050873,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM1069208,0,40.0,0.828,0.152,0.452,0.056,0.559,0.250,0.840,0.212,...,0.232,0.898,0.038,0.083,0.073,0.044,0.023,0.231,0.193,0.602
GSM1443533,0,66.0,0.882,0.270,0.382,0.183,0.652,0.072,0.874,0.376,...,0.121,0.955,0.047,0.136,0.097,0.099,0.033,0.284,0.232,0.562
GSM1443480,0,68.0,NaN,0.374,0.363,0.122,0.575,0.171,0.881,0.434,...,0.109,0.956,0.052,0.240,0.115,0.082,0.089,0.301,0.234,0.541
GSM2139236,0,70.0,0.845,0.264,0.500,0.125,0.623,0.092,0.811,0.333,...,0.092,0.928,0.056,0.124,0.144,0.085,0.111,0.288,0.214,0.450
GSM1069349,0,72.0,NaN,0.378,0.466,0.111,0.538,0.089,0.845,0.433,...,0.078,0.962,0.058,0.208,0.062,0.090,0.082,NaN,0.292,0.632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871865,1,67.0,NaN,0.289,0.567,0.130,0.590,0.203,0.776,0.410,...,0.208,0.948,0.029,0.151,0.094,0.065,0.064,0.252,0.159,0.489
GSM1871627,1,51.0,NaN,0.328,0.427,0.116,0.603,0.202,0.898,0.348,...,0.148,0.956,0.045,0.161,0.082,0.061,0.057,0.302,0.223,0.535
GSM1871629,1,62.0,NaN,0.232,0.356,0.050,0.653,0.216,0.872,0.284,...,0.189,0.981,0.054,0.199,0.057,0.050,0.049,0.218,0.187,0.420
GSM1871734,1,71.0,NaN,0.330,0.351,0.135,0.610,0.233,0.907,0.353,...,0.246,0.953,0.073,0.149,0.106,0.048,0.062,0.316,0.240,0.543


In [22]:
test_data=test_data.loc[test_data['AGE'] >= 20]
test_data=test_data.loc[test_data['AGE'] <= 110]
test_data

,status,AGE,cg00050873,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM1069208,0,40.0,0.828,0.152,0.452,0.056,0.559,0.250,0.840,0.212,...,0.232,0.898,0.038,0.083,0.073,0.044,0.023,0.231,0.193,0.602
GSM1443533,0,66.0,0.882,0.270,0.382,0.183,0.652,0.072,0.874,0.376,...,0.121,0.955,0.047,0.136,0.097,0.099,0.033,0.284,0.232,0.562
GSM1443480,0,68.0,NaN,0.374,0.363,0.122,0.575,0.171,0.881,0.434,...,0.109,0.956,0.052,0.240,0.115,0.082,0.089,0.301,0.234,0.541
GSM2139236,0,70.0,0.845,0.264,0.500,0.125,0.623,0.092,0.811,0.333,...,0.092,0.928,0.056,0.124,0.144,0.085,0.111,0.288,0.214,0.450
GSM1069349,0,72.0,NaN,0.378,0.466,0.111,0.538,0.089,0.845,0.433,...,0.078,0.962,0.058,0.208,0.062,0.090,0.082,NaN,0.292,0.632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871865,1,67.0,NaN,0.289,0.567,0.130,0.590,0.203,0.776,0.410,...,0.208,0.948,0.029,0.151,0.094,0.065,0.064,0.252,0.159,0.489
GSM1871627,1,51.0,NaN,0.328,0.427,0.116,0.603,0.202,0.898,0.348,...,0.148,0.956,0.045,0.161,0.082,0.061,0.057,0.302,0.223,0.535
GSM1871629,1,62.0,NaN,0.232,0.356,0.050,0.653,0.216,0.872,0.284,...,0.189,0.981,0.054,0.199,0.057,0.050,0.049,0.218,0.187,0.420
GSM1871734,1,71.0,NaN,0.330,0.351,0.135,0.610,0.233,0.907,0.353,...,0.246,0.953,0.073,0.149,0.106,0.048,0.062,0.316,0.240,0.543


In [23]:
test_data.to_csv('brain_55_cpgs_test.csv', encoding='utf-8', index=True)

In [24]:
test_data=pd.read_csv('brain_55_cpgs_test.csv', index_col=0,low_memory=False)

In [25]:
train_data=pd.read_csv('brain_55_cpgs_train.csv', index_col=0,low_memory=False)
train_data

,status,AGE,cg00050873,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM2139432,0,71.0,0.826,0.288,0.331,0.128,0.607,0.106,0.829,0.299,...,0.088,0.948,0.037,0.217,0.105,0.102,0.124,0.289,0.240,0.475
GSM2139249,0,76.0,NaN,0.329,0.378,0.140,0.563,0.101,0.880,0.413,...,0.095,0.942,0.039,0.161,0.130,0.064,0.084,0.318,0.215,0.466
GSM2139398,0,102.0,NaN,0.294,0.339,0.067,0.604,0.188,0.824,0.395,...,0.154,0.947,0.044,0.124,0.114,0.085,0.063,0.317,0.219,0.432
GSM2139297,0,108.0,NaN,0.327,0.528,0.153,0.703,0.054,0.847,0.438,...,0.053,0.964,0.088,0.261,0.168,0.105,0.192,0.419,0.293,0.559
GSM1443489,0,40.0,0.828,0.152,0.452,0.056,0.559,0.250,0.840,0.212,...,0.232,0.898,0.038,0.083,0.073,0.044,0.023,0.231,0.193,0.602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871717,1,42.0,NaN,0.209,0.464,0.056,0.650,0.198,0.846,0.109,...,0.198,0.920,0.041,0.079,0.048,0.028,0.033,0.158,0.194,0.518
GSM1871849,1,62.0,NaN,0.212,0.433,0.125,0.667,0.044,0.845,0.155,...,0.062,0.969,0.049,0.141,0.066,0.044,0.063,0.205,0.211,0.420
GSM1871852,1,58.0,NaN,0.250,0.449,0.120,0.623,0.212,0.917,0.273,...,0.159,0.935,0.035,0.141,0.069,0.048,0.043,0.233,0.176,0.496
GSM1871860,1,91.0,NaN,0.287,0.657,0.129,0.649,0.017,0.902,0.324,...,0.055,0.983,0.052,0.154,0.069,0.060,0.057,0.276,0.237,0.384


In [26]:
train_data=train_data.loc[train_data['status'] != 2]
train_data

,status,AGE,cg00050873,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM2139432,0,71.0,0.826,0.288,0.331,0.128,0.607,0.106,0.829,0.299,...,0.088,0.948,0.037,0.217,0.105,0.102,0.124,0.289,0.240,0.475
GSM2139249,0,76.0,NaN,0.329,0.378,0.140,0.563,0.101,0.880,0.413,...,0.095,0.942,0.039,0.161,0.130,0.064,0.084,0.318,0.215,0.466
GSM2139398,0,102.0,NaN,0.294,0.339,0.067,0.604,0.188,0.824,0.395,...,0.154,0.947,0.044,0.124,0.114,0.085,0.063,0.317,0.219,0.432
GSM2139297,0,108.0,NaN,0.327,0.528,0.153,0.703,0.054,0.847,0.438,...,0.053,0.964,0.088,0.261,0.168,0.105,0.192,0.419,0.293,0.559
GSM1443489,0,40.0,0.828,0.152,0.452,0.056,0.559,0.250,0.840,0.212,...,0.232,0.898,0.038,0.083,0.073,0.044,0.023,0.231,0.193,0.602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871717,1,42.0,NaN,0.209,0.464,0.056,0.650,0.198,0.846,0.109,...,0.198,0.920,0.041,0.079,0.048,0.028,0.033,0.158,0.194,0.518
GSM1871849,1,62.0,NaN,0.212,0.433,0.125,0.667,0.044,0.845,0.155,...,0.062,0.969,0.049,0.141,0.066,0.044,0.063,0.205,0.211,0.420
GSM1871852,1,58.0,NaN,0.250,0.449,0.120,0.623,0.212,0.917,0.273,...,0.159,0.935,0.035,0.141,0.069,0.048,0.043,0.233,0.176,0.496
GSM1871860,1,91.0,NaN,0.287,0.657,0.129,0.649,0.017,0.902,0.324,...,0.055,0.983,0.052,0.154,0.069,0.060,0.057,0.276,0.237,0.384


In [27]:
test_data=test_data.loc[test_data['status'] !=2]
test_data

,status,AGE,cg00050873,cg00807959,cg01066472,cg13806070,cg15907146,cg17104258,cg24441324,cg22454769,...,cg19622662,cg23595055,cg04739123,cg16367511,cg18008766,cg19451698,cg04834794,cg07303143,cg21182694,cg23352942
GSM1069208,0,40.0,0.828,0.152,0.452,0.056,0.559,0.250,0.840,0.212,...,0.232,0.898,0.038,0.083,0.073,0.044,0.023,0.231,0.193,0.602
GSM1443533,0,66.0,0.882,0.270,0.382,0.183,0.652,0.072,0.874,0.376,...,0.121,0.955,0.047,0.136,0.097,0.099,0.033,0.284,0.232,0.562
GSM1443480,0,68.0,NaN,0.374,0.363,0.122,0.575,0.171,0.881,0.434,...,0.109,0.956,0.052,0.240,0.115,0.082,0.089,0.301,0.234,0.541
GSM2139236,0,70.0,0.845,0.264,0.500,0.125,0.623,0.092,0.811,0.333,...,0.092,0.928,0.056,0.124,0.144,0.085,0.111,0.288,0.214,0.450
GSM1069349,0,72.0,NaN,0.378,0.466,0.111,0.538,0.089,0.845,0.433,...,0.078,0.962,0.058,0.208,0.062,0.090,0.082,NaN,0.292,0.632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM1871865,1,67.0,NaN,0.289,0.567,0.130,0.590,0.203,0.776,0.410,...,0.208,0.948,0.029,0.151,0.094,0.065,0.064,0.252,0.159,0.489
GSM1871627,1,51.0,NaN,0.328,0.427,0.116,0.603,0.202,0.898,0.348,...,0.148,0.956,0.045,0.161,0.082,0.061,0.057,0.302,0.223,0.535
GSM1871629,1,62.0,NaN,0.232,0.356,0.050,0.653,0.216,0.872,0.284,...,0.189,0.981,0.054,0.199,0.057,0.050,0.049,0.218,0.187,0.420
GSM1871734,1,71.0,NaN,0.330,0.351,0.135,0.610,0.233,0.907,0.353,...,0.246,0.953,0.073,0.149,0.106,0.048,0.062,0.316,0.240,0.543


## Model

In [28]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [30]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

In [137]:
def mean_impute(data):
    nas=data.isnull().sum()    
    col_means=data.mean(axis=0)
    na_cols=[]
    na_cols_means=[]

    for i in range(len(nas)):
        if nas[i]!=0:        
            na_cols.append(nas.keys()[i])
            na_cols_means.append(col_means[i])
        
    ids=list(data.index)
    for i in ids:
        for j in range(len(na_cols)):
            if str(data.loc[i][na_cols[j]])=="nan":
                data.loc[i][na_cols[j]]=na_cols_means[j]
    
    
    return data

In [138]:
model = Sequential()
model.add(Dense(12, input_dim=56, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [139]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [140]:
Xwork = train_data.iloc[:,2:]
ywork = train_data.iloc[:,0]

Xwork = mean_impute(Xwork)

In [151]:
Xtest = test_data.iloc[:,2:]
ytest = test_data.iloc[:,0]

Xtest = mean_impute(Xtest)

In [152]:
model.fit(Xwork, ywork, epochs=60, batch_size=10, verbose=0)

In [153]:
_, accuracy = model.evaluate(Xwork, ywork)
print('Accuracy: %.2f' % (accuracy*100))

49/49 [==============================] - 0s 424us/step - loss: 0.4694 - accuracy: 0.7417
Accuracy: 74.17


In [154]:
ypred=model.predict_classes(Xtest)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [155]:
ytest

GSM1069208    0
GSM1443533    0
GSM1443480    0
GSM2139236    0
GSM1069349    0
             ..
GSM2809053    1
GSM2809058    1
GSM2809061    1
GSM2809063    1
GSM2809064    1
Name: status, Length: 507, dtype: int64

In [157]:
print(classification_report(ytest,ypred,target_names=['Healthy','Alzheimers']))

              precision    recall  f1-score   support

     Healthy       0.77      0.84      0.80       313
  Alzheimers       0.70      0.60      0.65       194

    accuracy                           0.75       507
   macro avg       0.73      0.72      0.73       507
weighted avg       0.74      0.75      0.74       507

